In [ ]:
!python pip install requests

In [ ]:
import json
import urllib.parse
import requests
from datasets import load_dataset

# Retrieve Wikipedia articles matching a topic

There doesn't seem to be a very easy way to get the full articles that are recursively related to a category.
There is a tool called PetScan https://petscan.wmflabs.org/ that can find all related articles but it only returns ids, title and not the content.

There is an API to get the full page but we would likely need to call it too many times.

There is a special export UI that lets you list articles but it doesn't seem well suited to this usecase.

What we will do is filter a datascience Wikipedia dump for the matching articles.

In [ ]:

# Provide the exact wikipedia category name 
category = "Military equipment"
# The depth controls how deep the crawl goes
depth = 3

# We need to plus encode the category for the url
category = urllib.parse.quote_plus(category)
url =f'https://petscan.wmflabs.org/?depth={depth}&format=json&regexp_filter=&links_to_no=&cb_labels_any_l=1&search_query=&sparql=&ores_type=any&output_compatability=catscan&langs_labels_no=&max_sitelink_count=&templates_no=&search_max_results=500&wpiu=any&ores_prob_to=&referrer_name=&project=wikipedia&templates_any=&ns%5B0%5D=1&categories={category}&links_to_all=&ores_prediction=any&labels_no=&wikidata_item=no&interface_language=en&namespace_conversion=keep&manual_list_wiki=&sortby=none&show_soft_redirects=both&search_wiki=&wikidata_source_sites=&edits%5Bflagged%5D=both&labels_yes=&after=&cb_labels_yes_l=1&min_redlink_count=1&larger=&edits%5Banons%5D=both&common_wiki_other=&cb_labels_no_l=1&language=en&page_image=any&doit='
response =  requests.get(url,headers={'Accept': 'application/json'})
data = response.json()

articles=data['*'][0]['a']['*']

print(f'Found {len(articles)} articles from Pet Scan  for Category:{category} and depth:{depth}')

In [ ]:
print(f'Found {len(articles)} articles from Pet Scan  for Category:{category} and depth:{depth}')

In [ ]:
# We just need the list of IDs so we can filter the main dataset
articles=data['*'][0]['a']['*']
ids=[]
for article in articles:
    ids.append(str(article['id']))


In [ ]:
# Load the whole wikipedia dataset from the Hugging Face cleaned Dataset
# This will take a while to download the ~20GB but it is then locally cached for this user
wiki=load_dataset("wikipedia", "20220301.en")


In [ ]:

# This is obviously not very efficient and there maybe a better way to do this 
# In my testing this was taking about 1h

local_filename = f'./wiki_articles_category_{category}_{depth}'

wiki_articles = wiki['train']
try:
    wiki_match_category.load_from_disk(local_filename)
    print('Sucessfully loaded data from disk')
except:
    print('Filtering the dataset')
    wiki_match_category = wiki_articles.filter(lambda x: x['id'] in ids)
    wiki_match_category.save_to_disk(local_filename)

In [ ]:
number_of_docs = len(wiki_match_category)
print(f'Extracted and saved {number_of_docs} articles')
print(f'This maybe slightly different from the original {len(articles)} due to a missmatch in the current version and the historic, cleaned data')